# Dynet segmentation with tf fold
![animation](../../fold/tensorflow_fold/g3doc/animation.gif)  

In [1]:
#just a bunch of fun
import numpy as np
import six
import time
from multiprocessing import Process, Queue
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
BATCH_SIZE = 100
data_dir = "/home/moon/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]



[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'D', 'E', 'G', 'L', 'N', 'S', 'T', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [2]:
class data():
    def __init__(self, folder, truncate=200):
        self.data_dir = folder
        self.data = dict()
        self.size = dict()
        self.datasets = ["train", "test", "validation"]
        self.truncate = truncate
        
        for dataset in self.datasets:
            self.data[dataset] = self.sentence_reader(folder+dataset)
            #self.size[dataset] = sum(1 for line in open(folder+dataset))

                        
    def sentence_reader(self, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            i=0
            sentence = []
            end_sentence = False
            with open(file) as f:
                for lines in f:
                    line = lines[:-1].split('\t')
                    if line[0] != "":
                        sentence.append(line)
                    else:
                        end_sentence = True
                        i+=1
                    if end_sentence:
                        end_sentence = False
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        if len(sent) != 0:
                            sent_onehot = self.onehot(sent)
                            if len(sent_onehot) == len(tags):
                                if len(sent_onehot) >= self.truncate:
                                    sent_onehot=sent_onehot[:self.truncate]
                                    tags = tags[:self.truncate]
                                yield (sent_onehot, tags)
                            sentence = []      
          
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        indices = np.arange(len(string)), np.array([int(index(char)) for char in string])
        onehot[indices]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
store = data("/home/moon/data/")

## helper functions

In [3]:
def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]


In [4]:
!nvidia-smi

Sun Oct 15 18:42:00 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.66                 Driver Version: 375.66                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 850M    Off  | 0000:01:00.0     Off |                  N/A |
| N/A   44C    P0    N/A /  N/A |    347MiB /  4044MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [88]:
def convLSTM_cell(kernel_size, out_features = 64):
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=out_features, kernel_shape=[kernel_size])
    return td.ScopedLayer(convlstm)

def multi_convLSTM_cell(kernel_sizes, out_features):
    stacked_convLSTM = tf.contrib.rnn.MultiRNNCell()
    return td.ScopedLayer(stacked_convLSTM)
    
def bidirectional_dynamic_nn(fw_cell, bw_cell, out_features=64):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = (td.Function(lambda x: tf.reshape(x, [-1,vsize*out_features])) >>
                            #td.FC(10, activation=tf.nn.tanh) >>
                            td.FC(1, activation=None))
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)
                    
        #tag_logits = td.Map(output_transform).reads(bidir_common)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm


data = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))

model =  data >> bidirectional_dynamic_nn(convLSTM_cell(vsize), convLSTM_cell(vsize)) >> td.Void()
labels = td.Map(td.Scalar() >> td.Metric("labels")) >> td.Void()

In [89]:
compiler = td.Compiler.create((model, labels))
logits = tf.squeeze(compiler.metric_tensors['logits'])
labels = compiler.metric_tensors['labels']

loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
l2_loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
log_loss = (labels) * tf.log(logits) + (1 - labels) * tf.log(1 - logits)
cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels)

preds = tf.nn.sigmoid(logits)

opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(cross_entropy_loss)
sess.run(tf.global_variables_initializer())


/home/moon/Envs/tf1/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [90]:
sess.run(tf.global_variables_initializer())
feed = compiler.build_feed_dict([(onehot("naGon jó ötlet"),[0,0,0,1,0,1,0,0,0,0,0,1,0,0]) for i in range(1)])
#feed = compiler.build_feed_dict([next(store.data["train"]) for _ in range(5)])
for i in range(1000):
    a,b,c,d= sess.run([preds, compiler.metric_tensors['labels'], loss, train_op], feed)
    print("preds: ", a)
    print("labels: ", b)
    print("loss: ", c, '\n')

preds:  [ 0.49919888  0.49990645  0.49973264  0.49942607  0.49873754  0.49914899
  0.49948591  0.4984366   0.49840343  0.49972168  0.50046206  0.49999994
  0.5002861   0.50135708]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  0.217116 

preds:  [ 0.49515361  0.49227434  0.48997447  0.4892112   0.4838289   0.48241836
  0.48069695  0.47907615  0.47741219  0.47671202  0.47768885  0.47857046
  0.47922811  0.48005807]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  0.282234 

preds:  [ 0.49100962  0.48427814  0.4793098   0.47735184  0.46629626  0.46189764
  0.45674053  0.44877899  0.44260263  0.43690658  0.43477175  0.43344861
  0.43083635  0.42775944]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  0.394508 

preds:  [ 0.48676783  0.47554559  0.46633011  0.46046159  0.4397296   0.42649624
  0.40985617  0.34896579  0.29725829  0.22804216  0.14632814  0.06559449
  0.01670568  0.00275557]
labels:  [ 0.  0.  0.  1.

preds:  [ 0.42355067  0.39822334  0.4333539   0.57403445  0.64540952  0.84039223
  0.10922459  0.18160409  0.11412781  0.23833835  0.12394835  0.38081729
  0.16081108  0.35568357]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  1.10388 

preds:  [ 0.42003509  0.39408383  0.42926219  0.57840705  0.64571363  0.84875566
  0.0296584   0.17783748  0.07813743  0.26622123  0.08830313  0.35634962
  0.10356664  0.30528057]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  1.31811 

preds:  [ 0.41623649  0.38920566  0.42018333  0.56649941  0.59051317  0.86555666
  0.02791123  0.14054209  0.05004479  0.19662394  0.03970053  0.28072017
  0.04403438  0.2042629 ]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  1.59631 

preds:  [ 0.41219661  0.38457978  0.40864813  0.55546468  0.51578456  0.87029147
  0.04307976  0.1164179   0.03538292  0.16815789  0.01875879  0.30070546
  0.01687146  0.21799336]
labels:  [ 0.  0.  0.  1.  0

preds:  [  3.15903783e-01   3.55082899e-01   2.86120427e-04   9.98989999e-01
   1.17761978e-07   9.99824703e-01   8.74856525e-08   1.21967867e-02
   1.26520293e-02   3.91641706e-02   2.28840545e-01   4.22948986e-01
   4.42592800e-02   8.38474603e-04]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  5.70807 

preds:  [  3.13166350e-01   3.32290769e-01   2.92190147e-04   9.98849392e-01
   1.09355085e-07   9.99810278e-01   8.02603708e-08   8.27360060e-03
   7.08439713e-03   2.12098379e-02   1.70756310e-01   4.40921813e-01
   4.77040298e-02   5.88600407e-04]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  5.86665 

preds:  [  3.10473293e-01   3.09769928e-01   3.05929949e-04   9.98792768e-01
   1.02530116e-07   9.99809682e-01   7.37809671e-08   5.03573613e-03
   3.17951222e-03   8.66378099e-03   1.20431669e-01   6.24009490e-01
   1.90592811e-01   1.54479174e-03]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  5.833

preds:  [  2.44469017e-01   6.18423000e-02   4.53984721e-05   9.99970198e-01
   8.34785965e-08   9.99976873e-01   7.01640772e-08   3.17874656e-04
   1.31464512e-05   5.89931460e-06   1.86795101e-03   9.15684521e-01
   7.96915963e-02   1.44201647e-06]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  8.61506 

preds:  [  2.41863221e-01   5.90194948e-02   3.58660873e-05   9.99973178e-01
   8.70512125e-08   9.99979973e-01   7.37832266e-08   4.42168937e-04
   1.80729203e-05   3.42915655e-06   2.16721688e-04   6.59990966e-01
   6.33739114e-01   2.32972397e-05]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  8.38484 

preds:  [  2.39270002e-01   5.61064072e-02   2.85441420e-05   9.99972582e-01
   9.11887028e-08   9.99980330e-01   7.76924054e-08   3.88937391e-04
   1.60652889e-05   4.40679105e-06   5.53647405e-04   7.74521768e-01
   1.74483910e-01   3.00702391e-06]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  8.542

preds:  [  1.84701025e-01   1.37705784e-02   6.02742318e-08   9.99953032e-01
   2.99078238e-08   9.99991059e-01   2.76860259e-08   1.71690670e-04
   2.15762520e-06   3.36424108e-07   3.77015589e-04   9.91218269e-01
   3.20711872e-03   1.32474405e-08]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  10.6288 

preds:  [  1.82793438e-01   1.30112031e-02   4.65315644e-08   9.99949574e-01
   2.80957977e-08   9.99991179e-01   2.60717901e-08   1.74056870e-04
   2.10295593e-06   2.86187884e-07   2.87493662e-04   9.91847754e-01
   5.52412262e-03   1.38777452e-08]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  10.6518 

preds:  [  1.80912003e-01   1.23002715e-02   3.60134322e-08   9.99945641e-01
   2.64281699e-08   9.99991298e-01   2.45790854e-08   1.73572771e-04
   2.01134117e-06   2.50009435e-07   2.44779716e-04   9.92864966e-01
   7.93055445e-03   1.40530121e-08]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  10.68

...

preds:  [  2.91578029e-03   1.76048882e-08   7.03514712e-18   9.99999642e-01
   2.13321756e-11   1.00000000e+00   1.67578434e-11   3.04905734e-06
   9.57171675e-10   2.57514982e-11   8.96606161e-06   9.99990702e-01
   3.66838412e-06   1.50857126e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.7331 

preds:  [  2.90826964e-03   1.74725177e-08   6.99780902e-18   9.99999642e-01
   2.12482410e-11   1.00000000e+00   1.66926056e-11   3.04420814e-06
   9.54236912e-10   2.56494877e-11   8.94710774e-06   9.99990702e-01
   3.66080371e-06   1.50315990e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.7364 

preds:  [  2.90079275e-03   1.73412094e-08   6.96069590e-18   9.99999642e-01
   2.11645944e-11   1.00000000e+00   1.66277807e-11   3.03937259e-06
   9.51312917e-10   2.55477358e-11   8.92823664e-06   9.99990702e-01
   3.65324240e-06   1.49777673e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.73

preds:  [  2.72927387e-03   1.45081200e-08   6.14054343e-18   9.99999642e-01
   1.92848463e-11   1.00000000e+00   1.51680647e-11   2.92739696e-06
   8.84736062e-10   2.32608117e-11   8.49280332e-06   9.99991179e-01
   3.47900118e-06   1.37608176e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.8178 

preds:  [  2.72244890e-03   1.44022492e-08   6.10907169e-18   9.99999642e-01
   1.92115681e-11   1.00000000e+00   1.51110912e-11   2.92288883e-06
   8.82107443e-10   2.31714977e-11   8.47540832e-06   9.99991179e-01
   3.47201762e-06   1.37131322e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.821 

preds:  [  2.71564536e-03   1.42972301e-08   6.07783032e-18   9.99999642e-01
   1.91385310e-11   1.00000000e+00   1.50543605e-11   2.91840161e-06
   8.79486706e-10   2.30824821e-11   8.45803243e-06   9.99991179e-01
   3.46506135e-06   1.36656127e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.824

preds:  [  2.55943160e-03   1.20243975e-08   5.38500865e-18   9.99999642e-01
   1.74934338e-11   1.00000000e+00   1.37756898e-11   2.81427060e-06
   8.19763923e-10   2.10759292e-11   8.05727905e-06   9.99991655e-01
   3.30447142e-06   1.25898066e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.8999 

preds:  [  2.55320407e-03   1.19391093e-08   5.35836972e-18   9.99999642e-01
   1.74290547e-11   1.00000000e+00   1.37255936e-11   2.81007055e-06
   8.17401646e-10   2.09973653e-11   8.04123465e-06   9.99991655e-01
   3.29802378e-06   1.25474945e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.903 

preds:  [  2.54700007e-03   1.18545618e-08   5.33190366e-18   9.99999642e-01
   1.73651128e-11   1.00000000e+00   1.36758660e-11   2.80588779e-06
   8.15046142e-10   2.09191362e-11   8.02524664e-06   9.99991655e-01
   3.29159798e-06   1.25054437e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.906

preds:  [  2.40434171e-03   1.00193374e-08   4.74325216e-18   9.99999642e-01
   1.59190230e-11   1.00000000e+00   1.25508050e-11   2.70881014e-06
   7.61280317e-10   1.91514218e-11   7.65556251e-06   9.99992013e-01
   3.14316958e-06   1.15504340e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.9795 

preds:  [  2.39864807e-03   9.95031968e-09   4.72056218e-18   9.99999642e-01
   1.58623739e-11   1.00000000e+00   1.25066676e-11   2.70489431e-06
   7.59148910e-10   1.90820710e-11   7.64074957e-06   9.99992013e-01
   3.13721034e-06   1.15128458e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.9825 

preds:  [  2.39297491e-03   9.88183491e-09   4.69801654e-18   9.99999642e-01
   1.58059902e-11   1.00000000e+00   1.24627808e-11   2.70099667e-06
   7.57026275e-10   1.90130429e-11   7.62593527e-06   9.99992132e-01
   3.13127703e-06   1.14753996e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  18.98

preds:  [  2.26237183e-03   8.39103720e-09   4.19519289e-18   9.99999762e-01
   1.45297810e-11   1.00000000e+00   1.14689291e-11   2.61028731e-06
   7.08470227e-10   1.74498783e-11   7.28406712e-06   9.99992490e-01
   2.99376120e-06   1.06245752e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  19.0568 

preds:  [  2.25715246e-03   8.33477820e-09   4.17574587e-18   9.99999762e-01
   1.44796787e-11   1.00000000e+00   1.14299264e-11   2.60663046e-06
   7.06541881e-10   1.73884136e-11   7.27031920e-06   9.99992490e-01
   2.98824489e-06   1.05910495e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  19.0597 

preds:  [  2.25195312e-03   8.27895974e-09   4.15642086e-18   9.99999762e-01
   1.44298037e-11   1.00000000e+00   1.13910348e-11   2.60297884e-06
   7.04621417e-10   1.73273305e-11   7.25661766e-06   9.99992490e-01
   2.98274131e-06   1.05576095e-12]
labels:  [ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
loss:  19.06

In [ ]:
sess.run(tf.global_variables_initializer())
data = [next(store.data["train"]) for _ in range(1)]
print(data)
feed = compiler.build_feed_dict(data)
for i in range(1000):
    a,b,c,d= sess.run([preds, compiler.metric_tensors['labels'], loss, train_op], feed)
    print("preds: ", a)
    print("labels: ", b)
    print("loss: ", c, '\n')

[([array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.

preds:  [ 0.49983853  0.49886212  0.49881953  0.49974716  0.4989883   0.49898273
  0.49948114  0.49981838  0.50044912  0.50014818  0.49982077  0.49927935
  0.49890092  0.49883381  0.49891138  0.49960372  0.49900979  0.49986711
  0.49906012  0.49902961  0.49739015  0.49736619  0.49862289  0.49849969
  0.49919009  0.5001207   0.50002992  0.50040078  0.49967536  0.49942881
  0.49912593  0.49953452  0.49971536  0.4990871   0.49830726  0.49820343
  0.49807954  0.49802282  0.49873969]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  0.131826 

preds:  [ 0.49747118  0.49462941  0.4910762   0.48888406  0.48558703  0.48264813
  0.48073244  0.47912902  0.47825032  0.47783852  0.47614315  0.475851
  0.47587502  0.47668689  0.47620532  0.47631168  0.4761979   0.47716281
  0.47644797  0.47560182  0.46741223  0.46666104  0.46766728  0.46744224
  0.46768558  0.46875098  0.468

preds:  [ 0.48873413  0.48037797  0.4644388   0.44944674  0.43294999  0.41225347
  0.39168274  0.36948591  0.34520683  0.32213816  0.28855565  0.25843433
  0.2235027   0.18848231  0.14780788  0.11143117  0.08214476  0.05838744
  0.04117711  0.02891904  0.16292708  0.17013918  0.16558024  0.17083912
  0.16481718  0.16603494  0.16586015  0.1661872   0.17078464  0.16664739
  0.17185777  0.16684833  0.16685946  0.17150891  0.16572286  0.17272265
  0.17202953  0.16728701  0.16634345]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  1.51808 

preds:  [ 0.48809606  0.4792752   0.46248764  0.44662493  0.42911154  0.40695709
  0.38467872  0.36033621  0.33330545  0.30672812  0.26843482  0.23298986
  0.1923297   0.15245171  0.11006436  0.07572112  0.05118194  0.0339807
  0.02315183  0.01627271  0.14467937  0.15165064  0.14692688  0.15200739
  0.14577585  0.14681293  0.146

preds:  [ 0.47847262  0.46114153  0.43157116  0.40556133  0.37967363  0.34964994
  0.32596841  0.30657393  0.29203415  0.28392863  0.26344872  0.2529535
  0.23443297  0.21371631  0.17301007  0.13061555  0.09248902  0.05626795
  0.03011629  0.01371031  0.14961685  0.16676866  0.15486737  0.16773729
  0.15225446  0.15478349  0.15467666  0.15522502  0.16639566  0.15533859
  0.16868261  0.15492727  0.15463369  0.16724679  0.15306924  0.1700768
  0.16834436  0.15637115  0.15382747]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  1.61304 

preds:  [ 0.47748324  0.45902783  0.42775044  0.40028179  0.37332407  0.34270909
  0.31957585  0.30185291  0.29007953  0.28521633  0.26725456  0.25841218
  0.23860747  0.21300904  0.16264969  0.1099967   0.06546111  0.03085792
  0.0121851   0.00408453  0.14478837  0.16253607  0.15021561  0.1635935
  0.14755249  0.15019181  0.15015

preds:  [ 0.46801722  0.43874142  0.39023095  0.34678096  0.30691266  0.26848096
  0.246132    0.23741864  0.24242945  0.25321355  0.23947719  0.221448
  0.17786454  0.13145359  0.07848419  0.04469892  0.026495    0.01554873
  0.00966236  0.00604609  0.13530505  0.16361833  0.14396244  0.16562177
  0.13926938  0.14295629  0.14325027  0.14435691  0.16288085  0.14487346
  0.16771413  0.14430228  0.14373021  0.16544588  0.1425065   0.171581
  0.16849345  0.14831787  0.14431153]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  1.90708 

preds:  [ 0.46755141  0.4385263   0.39149138  0.35177779  0.3176719   0.28707349
  0.27443051  0.27822503  0.30000106  0.33472908  0.350959    0.37816161
  0.38884306  0.39816701  0.37622651  0.35176119  0.32438493  0.26665679
  0.17724194  0.07914679  0.134174    0.16362765  0.14322382  0.16582517
  0.13837664  0.1422161   0.142610

...

preds:  [  2.07547083e-01   4.97924946e-02   2.10968987e-03   1.05450981e-05
   7.06124625e-09   1.95131012e-11   2.29591303e-12   8.38200256e-12
   9.28648547e-10   3.99704686e-06   1.16580850e-05   9.99766409e-01
   1.35095270e-05   9.99802768e-01   1.42693509e-07   6.33486343e-05
   2.76689018e-13   2.07720757e-10   1.76346352e-15   2.47155196e-11
   3.11216689e-04   8.19683450e-08   7.28405372e-04   9.98724282e-01
   9.43628780e-04   2.30491096e-07   3.84925770e-05   2.05560427e-04
   1.91655472e-05   1.42812669e-05   2.67648895e-04   1.47902480e-04
   9.82239089e-06   6.66148139e-07   1.19212476e-04   9.99080062e-01
   9.99324441e-01   1.04562721e-04   9.62136539e-08]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  13.1541 

preds:  [  2.06989035e-01   4.94154915e-02   2.07307562e-03   1.01999813e-05
   6.72470302e-09   1.85798339e-11   2.20718756e-12   8

preds:  [  2.02020600e-01   4.61427532e-02   1.77031488e-03   7.55620613e-06
   4.33442970e-09   1.19763123e-11   1.55031923e-12   6.37701957e-12
   7.54379892e-10   3.66101312e-06   1.01896567e-05   9.99777734e-01
   1.27650555e-05   9.99813497e-01   1.37770940e-07   5.93246878e-05
   2.50082642e-13   1.68116993e-10   1.43130865e-15   2.04191924e-11
   2.38086126e-04   6.49725322e-08   7.61721341e-04   9.98914123e-01
   1.03979628e-03   2.13938449e-07   2.91173528e-05   1.85546043e-04
   2.31721133e-05   1.38481573e-05   2.04490527e-04   1.75103603e-04
   1.27644471e-05   5.75411207e-07   9.07937356e-05   9.99091744e-01
   9.99410152e-01   1.03100836e-04   8.62187193e-08]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  13.2828 

preds:  [  2.01474473e-01   4.57921475e-02   1.73947180e-03   7.30807187e-06
   4.12813161e-09   1.14084072e-11   1.49087914e-12   6

preds:  [  1.96612880e-01   4.27495018e-02   1.48458325e-03   5.40896599e-06
   2.66305067e-09   7.38413670e-12   1.05049617e-12   4.87228227e-12
   6.14074402e-10   3.36389576e-06   8.93315064e-06   9.99788344e-01
   1.20758677e-05   9.99823153e-01   1.32734201e-07   5.56673804e-05
   2.26157187e-13   1.37344275e-10   1.16644568e-15   1.70341467e-11
   2.29763973e-04   6.82194568e-08   9.01070540e-04   9.98893082e-01
   7.81582436e-04   1.96449434e-07   2.59967201e-05   1.12683745e-04
   1.86635625e-05   1.61093649e-05   1.83682248e-04   1.29353255e-04
   1.43575080e-05   7.94205846e-07   1.16898380e-04   9.99236107e-01
   9.99398589e-01   8.63014138e-05   7.61629693e-08]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  13.4018 

preds:  [  1.96078688e-01   4.24236916e-02   1.45863462e-03   5.23087965e-06
   2.53659715e-09   7.03742316e-12   1.01060122e-12   4

preds:  [  1.91324458e-01   3.95972468e-02   1.24428549e-03   3.86881766e-06
   1.63822822e-09   4.57509551e-12   7.14506687e-13   3.73999312e-12
   5.01228281e-10   3.10121959e-06   7.86256805e-06   9.99798357e-01
   1.14378126e-05   9.99831915e-01   1.27557840e-07   5.23301605e-05
   2.04468034e-13   1.13249667e-10   9.54246672e-16   1.43431239e-11
   2.38660446e-04   5.03498612e-08   7.12318055e-04   9.98953581e-01
   7.38106319e-04   1.90463624e-07   3.42503263e-05   1.27400664e-04
   1.39954336e-05   1.45299055e-05   2.12529572e-04   1.17575408e-04
   1.14676732e-05   6.52364065e-07   9.78832759e-05   9.99231339e-01
   9.99472201e-01   8.99331644e-05   7.52137908e-08]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  13.5263 

preds:  [  1.90802202e-01   3.92947122e-02   1.22247497e-03   3.74118918e-06
   1.56067015e-09   4.36247392e-12   6.87639127e-13   3

preds:  [  1.86154962e-01   3.66710648e-02   1.04240095e-03   2.76566857e-06
   1.00944875e-09   2.84947283e-12   4.87950609e-13   2.88520683e-12
   4.10405904e-10   2.86889303e-06   6.95036761e-06   9.99808013e-01
   1.08433915e-05   9.99839902e-01   1.22245723e-07   4.92900072e-05
   1.84725716e-13   9.42705636e-11   7.83649419e-16   1.21882062e-11
   2.45121162e-04   4.07440126e-08   6.70504174e-04   9.98968363e-01
   5.33964834e-04   1.44574642e-07   4.26473052e-05   1.54901703e-04
   1.05152822e-05   1.16606316e-05   2.54927669e-04   1.29600798e-04
   9.63007824e-06   5.79379162e-07   1.18903787e-04   9.99418736e-01
   9.99401331e-01   6.60975784e-05   7.09002208e-08]
labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  0.]
loss:  13.6498 

preds:  [  1.85644567e-01   3.63903232e-02   1.02408696e-03   2.67431460e-06
   9.61832947e-10   2.71850901e-12   4.69800251e-13   2